In [11]:
from src.first_model.preprocessing import PreProcessingFirstModel
from src.first_model.train import Modeling

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import math

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
PATH_RAW = "/home/pdsmoreau/data-science/tera/bncc-classifier-final/bncc-classifier/data/raw/bncc_first_classifier.csv"  # noqa
PATH_TO_SAVE = "/home/pdsmoreau/data-science/tera/bncc-classifier-final/bncc-classifier/data/curated/bncc_first_classifier_cleaned.csv"  # noqa

In [6]:
lista1 = ["a", "b"]
str(lista1)

"['a', 'b']"

In [7]:
prepro_obj = PreProcessingFirstModel(PATH_RAW)
frq_words, rare_words = prepro_obj.prepro()

# export list of frq and rare words on our data
# with open("frq_words.txt", "w") as f:
#     # print(frq_words)
#     _ = [f.write("%s\n" % item) for item in frq_words]

# with open("rare_words.txt", "w") as f:
#     # print(rare_words)
#     _ = [f.write("%s\n" % item) for item in rare_words]

# prepro_obj.export_cleaned_data(PATH_TO_SAVE)
modeling = Modeling(prepro_obj.data)
# # modeling.tokenization()
modeling.split_data()
# modeling.saving_pipeline()
# X_train_trans, X_test_trans = modeling.make_bag_of_words()
# modeling.train_evaluate_log_reg(
#     X_train_trans=X_train_trans, X_test_trans=X_test_trans
# )

In [8]:
modeling.X_train["questions_clean"]

77438    verificandose massa corporal cinco crianças re...
8211                                    sumô chegou brasil
5218     contudo ser humano produz coisas apesar terem ...
27150    enem 2010the weather manthey say british love ...
69111    arnaldo planeja ir praia deseja utilizar camis...
                               ...                        
27265    question editedconsiderando discussões atuais ...
28356    sobre micoses acometem seres humanos assinale ...
97160    leia trecho seguirpolítica valorização café co...
66343                                                lê 43
6878     novembro 2014 apenas dia registrados 18 casos ...
Name: questions_clean, Length: 56627, dtype: object

## Baseline BOW

In [9]:
# baselines
models = {
    "rg_lg": LogisticRegression(max_iter=500, n_jobs=8),
    "r_forest": RandomForestClassifier(max_depth=500, n_jobs=8),
    "lgbm": LGBMClassifier(n_jobs=8),
}

In [17]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
for train_idx, val_idx in kfold.split(
    modeling.X_train["questions_clean"], modeling.y_train
):
    for name, model in models.items():
        x_train, y_train = (
            modeling.X_train["questions_clean"].iloc[train_idx],
            modeling.y_train.iloc[train_idx],
        )
        x_val, y_val = (
            modeling.X_train["questions_clean"].iloc[val_idx],
            modeling.y_train.iloc[val_idx],
        )

        count_vectorizer = CountVectorizer()
        count_vectorizer.fit(x_train)
        X_train_cv = count_vectorizer.transform(x_train)
        X_val_cv = count_vectorizer.transform(x_val)

        if name == "lgbm":
            X_train_cv = X_train_cv.astype("float32")
            X_val_cv = X_val_cv.astype("float32")
            y_train = y_train.astype("float32")
            y_val = y_val.astype("float32")

        cv_classifier = model
        cv_classifier.fit(X_train_cv, y_train)
        y_pred = cv_classifier.predict(X_val_cv)
        f1 = f1_score(y_val, y_pred, average="macro")
        # print('ROC AUC - {}: {}'.format(name, mean))
        print("Model: {}. Macro avg F1: {}".format(name, f1))

0.7278253142975671
0.19407828326202928
0.6550434106278179
0.7145164707089338
0.7336190154838125
0.19210636960698546
0.649369269371921
0.7211956814872637
0.7330740839432095
0.19092967263959068
0.6498061041604263
0.7089284864570269
0.714314744373907
0.18948359277264393
0.6538362995130778
0.7173655668562345
0.7352967775129926
0.18483499341909196
0.655190162784483
0.7227555791757245


## Baseline N-Grams 2, 3

### Baseline N-grams only 2

In [7]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
for train_idx, val_idx in kfold.split(
    modeling.X_train["questions_clean"], modeling.y_train
):
    for name, model in models.items():
        x_train, y_train = (
            modeling.X_train["questions_clean"].iloc[train_idx],
            modeling.y_train.iloc[train_idx],
        )
        x_val, y_val = (
            modeling.X_train["questions_clean"].iloc[val_idx],
            modeling.y_train.iloc[val_idx],
        )

        count_vectorizer = CountVectorizer(ngram_range=(2, 2))
        count_vectorizer.fit(x_train)
        X_train_cv = count_vectorizer.transform(x_train)
        X_val_cv = count_vectorizer.transform(x_val)

        if name == "lgbm":
            X_train_cv = X_train_cv.astype("float32")
            X_val_cv = X_val_cv.astype("float32")
            y_train = y_train.astype("float32")
            y_val = y_val.astype("float32")

        cv_classifier = model
        cv_classifier.fit(X_train_cv, y_train)
        y_pred = cv_classifier.predict(X_val_cv)
        f1 = f1_score(y_val, y_pred, average="macro")
        # print('ROC AUC - {}: {}'.format(name, mean))
        print("Model: {}. Macro avg F1: {}".format(name, f1))

Model: rg_lg. Macro avg F1: 0.5986663758730688
Model: r_forest. Macro avg F1: 0.4763561365735009
Model: lgbm. Macro avg F1: 0.4672249081874527
Model: rg_lg. Macro avg F1: 0.5930171595626147
Model: r_forest. Macro avg F1: 0.47023744453816124
Model: lgbm. Macro avg F1: 0.45824420409745525
Model: rg_lg. Macro avg F1: 0.5900176617067225


KeyboardInterrupt: 

### Baseline N-grams with 1 and 2 grams

In [9]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
for train_idx, val_idx in kfold.split(
    modeling.X_train["questions_clean"], modeling.y_train
):
    for name, model in models.items():
        x_train, y_train = (
            modeling.X_train["questions_clean"].iloc[train_idx],
            modeling.y_train.iloc[train_idx],
        )
        x_val, y_val = (
            modeling.X_train["questions_clean"].iloc[val_idx],
            modeling.y_train.iloc[val_idx],
        )

        count_vectorizer = CountVectorizer(ngram_range=(1, 2))
        count_vectorizer.fit(x_train)
        X_train_cv = count_vectorizer.transform(x_train)
        X_val_cv = count_vectorizer.transform(x_val)

        if name == "lgbm":
            X_train_cv = X_train_cv.astype("float32")
            X_val_cv = X_val_cv.astype("float32")
            y_train = y_train.astype("float32")
            y_val = y_val.astype("float32")

        cv_classifier = model
        cv_classifier.fit(X_train_cv, y_train)
        y_pred = cv_classifier.predict(X_val_cv)
        f1 = f1_score(y_val, y_pred, average="macro")
        # print('ROC AUC - {}: {}'.format(name, mean))
        print("Model: {}. Macro avg F1: {}".format(name, f1))

Model: rg_lg. Macro avg F1: 0.7344466646738068
Model: r_forest. Macro avg F1: 0.6480102338994452
Model: lgbm. Macro avg F1: 0.7162755614846047
Model: rg_lg. Macro avg F1: 0.7342649079579414
Model: r_forest. Macro avg F1: 0.6370441908253599
Model: lgbm. Macro avg F1: 0.721983898464046


KeyboardInterrupt: 

## Word2Vec

In [11]:
from gensim.models import Word2Vec

In [13]:
modeling.X_train

,questions,target,questions_clean
71534,<p>Os diagramas PV a seguir representam o comp...,Física,"['diagramas', 'pv', 'seguir', 'representam', '..."
20097,"<p style=""text-align:justify;""> Um programa d...",Matemática,"['programa', 'computador', 'compactar', 'arqui..."
26713,"<p>Querendo pintar a nova fachada do Leme, Leo...",Física,"['querendo', 'pintar', 'nova', 'fachada', 'lem..."
43723,"<p><span style=""font-size:12pt;font-family:Ari...",Matemática,"['ef05ma06no', 'final', 'ano', 'passado', 'dir..."
18699,<p>Encaminhar foto pag 16</p>,Inglês,"['encaminhar', 'foto', 'pag', '16']"
...,...,...,...
26940,<p><div> Se os alunos observarem diariamente o...,Geografia,"['alunos', 'observarem', 'diariamente', 'nasce..."
28024,"<p>Observe a charge apresentada a seguir, que ...",Geografia,"['observe', 'charge', 'apresentada', 'seguir',..."
95998,<p>(FEI-SP) No movimento retil&iacute;neo unif...,Física,"['feisp', 'movimento', 'retilíneo', 'uniformem..."
65525,"<p>No circuito dessa quest&atilde;o, tr&ecirc;...",Física,"['circuito', 'dessa', 'questão', 'três', 'capa..."


In [14]:
wv = Word2Vec(
    sentences=modeling.X_train["questions_clean"].values,
    vector_size=100,
    window=3,
    min_count=1,
    workers=10,
)

In [15]:
def transforma_palavra(question):
    # Define dicionário para abrigar os vetores de palavras
    dict_vetores = {}
    # Usando a função get_vector, transforma a palavra no vetor de Embeddings
    for palavra in question:
        dict_vetores[palavra] = wv.wv.get_vector(palavra)
    return dict_vetores

In [17]:
transforma_palavra(modeling.X_train["questions_clean"].values[1])

{'[': array([-0.77741736,  0.2434256 ,  0.43531394,  0.01779383,  0.6338739 ,
        -0.32943684,  0.25893712, -0.21628594,  0.8445094 , -0.4462909 ,
        -0.52743167, -0.26064217,  0.01508226, -0.73761016, -0.20425954,
        -0.33216894,  0.683826  , -0.24963903,  0.56189   ,  0.0814601 ,
         0.0499045 , -0.3658407 ,  0.20846586,  0.11867454, -0.71989286,
         0.25772142, -0.06716782, -0.7419887 , -0.657276  , -0.36424172,
         0.992115  , -0.00226467,  0.28665945, -0.6022861 ,  0.1519862 ,
         0.5844902 ,  0.88316226,  0.6906269 ,  0.14161389,  0.20657212,
        -0.42842582, -0.9082406 , -0.25785768,  0.22815657,  0.03371271,
        -0.22049968,  0.11412928, -0.5643412 ,  0.49735478, -0.2807857 ,
        -0.7909623 , -0.520389  , -0.05012293,  0.32522714,  0.2261083 ,
         0.80260813,  0.16772021, -0.77698016, -1.0546687 ,  0.85921323,
         0.43889895, -0.51862   , -0.1472107 , -0.2981012 ,  1.1722546 ,
        -0.30417308, -0.6985332 , -0.8330787 ,

## Baseline TF-IDF Unigrams

In [12]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
for train_idx, val_idx in kfold.split(
    modeling.X_train["questions_clean"], modeling.y_train
):
    for name, model in models.items():
        x_train, y_train = (
            modeling.X_train["questions_clean"].iloc[train_idx],
            modeling.y_train.iloc[train_idx],
        )
        x_val, y_val = (
            modeling.X_train["questions_clean"].iloc[val_idx],
            modeling.y_train.iloc[val_idx],
        )
        tfidf = TfidfVectorizer()
        tfidf.fit(x_train)
        X_train_cv = tfidf.transform(x_train)
        X_val_cv = tfidf.transform(x_val)

        if name == "lgbm":
            X_train_cv = X_train_cv.astype("float32")
            X_val_cv = X_val_cv.astype("float32")
            y_train = y_train.astype("float32")
            y_val = y_val.astype("float32")

        # if name == "gnb":
        #     X_train_cv = X_train_cv.toarray()
        #     X_val_cv = X_val_cv.toarray()

        cv_classifier = model
        cv_classifier.fit(X_train_cv, y_train)
        y_pred = cv_classifier.predict(X_val_cv)
        f1 = f1_score(y_val, y_pred, average="macro")
        # print('ROC AUC - {}: {}'.format(name, mean))
        print("Model: {}. Macro avg F1: {}".format(name, f1))

Model: rg_lg. Macro avg F1: 0.7174284092094377
Model: r_forest. Macro avg F1: 0.662625236006448
Model: lgbm. Macro avg F1: 0.7300123508490808
Model: rg_lg. Macro avg F1: 0.7221191038100203
Model: r_forest. Macro avg F1: 0.6738584866971729
Model: lgbm. Macro avg F1: 0.7265470076520067
Model: rg_lg. Macro avg F1: 0.7092251363415872
Model: r_forest. Macro avg F1: 0.6530989358680449
Model: lgbm. Macro avg F1: 0.7137858421199376
Model: rg_lg. Macro avg F1: 0.7088298071040576
Model: r_forest. Macro avg F1: 0.6542158441048042
Model: lgbm. Macro avg F1: 0.7154126586073812
Model: rg_lg. Macro avg F1: 0.7116643414432898
Model: r_forest. Macro avg F1: 0.6632392909090893
Model: lgbm. Macro avg F1: 0.7217826281084121


## Baseline TF-IDF Unigrams and Bigrams

In [13]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
for train_idx, val_idx in kfold.split(
    modeling.X_train["questions_clean"], modeling.y_train
):
    for name, model in models.items():
        x_train, y_train = (
            modeling.X_train["questions_clean"].iloc[train_idx],
            modeling.y_train.iloc[train_idx],
        )
        x_val, y_val = (
            modeling.X_train["questions_clean"].iloc[val_idx],
            modeling.y_train.iloc[val_idx],
        )
        tfidf = TfidfVectorizer(ngram_range=(1, 2))
        count_vectorizer = CountVectorizer()
        tfidf.fit(x_train)
        X_train_cv = tfidf.transform(x_train)
        X_val_cv = tfidf.transform(x_val)

        if name == "lgbm":
            X_train_cv = X_train_cv.astype("float32")
            X_val_cv = X_val_cv.astype("float32")
            y_train = y_train.astype("float32")
            y_val = y_val.astype("float32")

        # if name == "gnb":
        #     X_train_cv = X_train_cv.toarray()
        #     X_val_cv = X_val_cv.toarray()

        cv_classifier = model
        cv_classifier.fit(X_train_cv, y_train)
        y_pred = cv_classifier.predict(X_val_cv)
        f1 = f1_score(y_val, y_pred, average="macro")
        # print('ROC AUC - {}: {}'.format(name, mean))
        print("Model: {}. Macro avg F1: {}".format(name, f1))

Model: rg_lg. Macro avg F1: 0.707041589011555
Model: r_forest. Macro avg F1: 0.6636466749229487
Model: lgbm. Macro avg F1: 0.727288312673762
Model: rg_lg. Macro avg F1: 0.7108201630598425
Model: r_forest. Macro avg F1: 0.6660251829630522
Model: lgbm. Macro avg F1: 0.7248634945379928
Model: rg_lg. Macro avg F1: 0.706018852026482
Model: r_forest. Macro avg F1: 0.6457269231999142
Model: lgbm. Macro avg F1: 0.7143485004165626
Model: rg_lg. Macro avg F1: 0.6977734746806653
Model: r_forest. Macro avg F1: 0.6469382868126617
Model: lgbm. Macro avg F1: 0.7146592351079871
Model: rg_lg. Macro avg F1: 0.698055281786434
Model: r_forest. Macro avg F1: 0.651408616771641
Model: lgbm. Macro avg F1: 0.7207498776743617
